# In the Name of God

# Imports:

In [1]:
# from typing import List
import math
import pandas as pd
import numpy as np
import hazm
# from hazm import Normalizer, word_tokenize, Stemmer, Lemmatizer, stopwords_list
import matplotlib.pyplot as plt

# Defines:

In [2]:
IGNORE_CASES = ["گزارش", "ایسنا"]

# Lemmatization vs Stemming:

In [3]:
stemmer = hazm.Stemmer()
print(stemmer.stem("می‌رفتم"))
lemmatizer = hazm.Lemmatizer()
print(lemmatizer.lemmatize("می‌رفتم").split("#")[0])

می‌رف
رفت


# Phase 1:

In [4]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

train_data
# train_data["content"][3]
i = 0
for index, row in train_data.iterrows():
    i += 1
    print(index)
    print(row)
    if i == 20:
        break

0
label                                                 فناوری
content    گزارش های منتشر شده حاکی از آن است که کاربران ...
Name: 0, dtype: object
1
label                                                  ورزشی
content    سوپر استار سینما و از قهرمانان سابق ووشو ــ کو...
Name: 1, dtype: object
2
label                                                  حوادث
content    مدیرعامل شرکت عمران آب کیش از فوت یک نفر در آت...
Name: 2, dtype: object
3
label                                                 فناوری
content    یک نوجوان انگلیسی به اتهام هک حساب‌های کاربری ...
Name: 3, dtype: object
4
label                                                  سلامت
content    دانشمندان در جدیدترین مطالعات خود اثرات جدید و...
Name: 4, dtype: object
5
label                                                  سلامت
content    مصرف میوه، یکی از راه های سالم برای کاهش وزن س...
Name: 5, dtype: object
6
label                                                  سیاسی
content    رئیس جمهوری پیش‌بینی، پیشگیری و درمان آسیب‌ه

In [5]:

class Parser:
    def __init__(self) -> None:
        self.normalizer = hazm.Normalizer()
        self.lemmatizer = hazm.Lemmatizer()
        self.tokenize = hazm.word_tokenize
        self.stop_words = hazm.stopwords_list()
        self.stop_words = self.stop_words + IGNORE_CASES

    def parse(self, text : str):
        tokens = self.tokenize(self.normalizer.normalize(text))
        tokens = [elem for elem in tokens if elem not in self.stop_words and elem.isalpha()]
        tokens = [self.lemmatize(elem) for elem in tokens if elem not in self.stop_words]
        return tokens

    def bigram_parse(self, text : str):
        tokens = self.tokenize(self.normalizer.normalize(text))
        tokens = [elem for elem in tokens if elem not in self.stop_words and elem.isalpha()]
        tokens = [self.lemmatize(elem) for elem in tokens if elem not in self.stop_words]
        bigram_tokens = []

        for i in range(len(tokens) - 1):
            bigram_tokens.append(tokens[i] + " " + tokens[i + 1])
        return bigram_tokens

    def lemmatize(self, word :str) -> str:
        return self.lemmatizer.lemmatize(word).split('#')[0]

p = Parser()
tokens = p.bigram_parse(train_data["content"][3])
print(tokens)
        

['نوجوان انگلیسی', 'انگلیسی اتهام', 'اتهام هک', 'هک کاربر', 'کاربر ایمیل', 'ایمیل ارشد', 'ارشد آمریکایی', 'آمریکایی سال', 'سال حبس', 'حبس دارالتادیب', 'دارالتادیب محکوم', 'محکوم نقل', 'نقل خبرگزاری', 'خبرگزاری آسوشیتدپرس', 'آسوشیتدپرس نوجوان', 'نوجوان نام', 'نام کین', 'کین گمبل', 'گمبل بازه', 'بازه زمان', 'زمان اقدام', 'اقدام هک', 'هک دسترس', 'دسترس اطلاعات', 'اطلاعات کاربر', 'کاربر مارک', 'مارک جیلیانو', 'جیلیانو مدیر', 'مدیر وقت', 'وقت اف', 'اف آی', 'آی جی', 'جی جانسون', 'جانسون وزیر', 'وزیر وقت', 'وقت کشور', 'کشور جان', 'جان برن', 'برن رئیس', 'رئیس وقت', 'وقت سیا', 'سیا گمبل', 'گمبل عضو', 'عضو گروه', 'گروه هکری', 'هکری موسوم', 'موسوم Crackas', 'Crackas With', 'With Attitude', 'Attitude اطلاعات', 'اطلاعات صورت', 'صورت آنلاین', 'آنلاین منتشر', 'منتشر گمبل', 'گمبل سال', 'سال گذشته', 'گذشته میلاد', 'میلاد گناهکار', 'گناهکار نوجوان', 'نوجوان انگلیسی', 'انگلیسی روز', 'روز جمعه', 'جمعه دادگاه', 'دادگاه کیفر', 'کیفر لندن', 'لندن سال', 'سال حبس', 'حبس دارالتادیب', 'دارالتادیب محکوم', 'محکوم 

In [6]:
class Topic:
    def __init__(self, name) -> None:
        self.name = name
        self.words = {}
    
    def add(self, tokens):
        for t in tokens:
            if t in self.words:
                self.words[t] += 1
            else:
                self.words.update({t: 1})        

topics = {}

def train():
    for index, row in train_data.iterrows():
        label = row["label"]
        content = p.bigram_parse(row["content"])

        if label not in topics:
            topics.update({label: Topic(label)})
        topics[label].add(content)    

train()

In [7]:
print(topics)

topics["فناوری"].words

{'فناوری': <__main__.Topic object at 0x7f7992ab4790>, 'ورزشی': <__main__.Topic object at 0x7f7992ab4ca0>, 'حوادث': <__main__.Topic object at 0x7f7992ab4910>, 'سلامت': <__main__.Topic object at 0x7f7993d33b20>, 'سیاسی': <__main__.Topic object at 0x7f7992ab6110>, 'فرهنگی/هنری': <__main__.Topic object at 0x7f7992ab4be0>}


{'منتشر حاکی': 11,
 'حاکی کاربر': 2,
 'کاربر تلگرام': 10,
 'تلگرام مناطق': 1,
 'مناطق جهان': 4,
 'جهان اروپا': 1,
 'اروپا خاورمیانه': 4,
 'خاورمیانه مشکل': 2,
 'مشکل نقل': 1,
 'نقل تلگرام': 1,
 'تلگرام دقایق': 1,
 'دقایق پیام': 1,
 'پیام رس': 54,
 'رس تلگرام': 3,
 'تلگرام صفحه': 1,
 'صفحه تو': 3,
 'تو اعلام': 6,
 'اعلام دسترس': 2,
 'دسترس کاربر': 36,
 'کاربر اپلیکیشن': 16,
 'اپلیکیشن اروپا': 1,
 'مشکل پیام': 2,
 'پیام منشا': 1,
 'منشا اختلالات': 1,
 'اختلالات ظهر': 1,
 'ظهر امروز': 4,
 'امروز شنبه': 6,
 'شنبه خارج': 1,
 'خارج پیام': 2,
 'رس ارتباط': 2,
 'ارتباط داخل': 4,
 'داخل کشور': 33,
 'کشور پیام': 1,
 'رس عنوان': 1,
 'عنوان متخصص': 2,
 'متخصص تلگرام': 1,
 'تلگرام زمان': 2,
 'زمان متوجه': 1,
 'متوجه اختلال': 1,
 'اختلال بررسی': 1,
 'بررسی برطرف': 1,
 'برطرف مشکل': 7,
 'مشکل مربوطه': 2,
 'مربوطه امید': 1,
 'امید زود': 1,
 'زود اختلال': 1,
 'اختلال رفع': 1,
 'رفع کاربر': 2,
 'کاربر کاربر': 25,
 'تلگرام دسترس': 2,
 'دسترس یافت': 3,
 'نوجوان انگلیسی': 2,
 'انگلیسی اتهام': 1,
 'اتهام هک